In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

2022-12-16 20:40:56.261642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 20:40:56.428668: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 20:40:56.428712: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 20:40:57.518748: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((592, 1), (148, 1), (592, 2), (148, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=200

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

2022-12-16 20:40:59.073245: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 20:40:59.073575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 20:40:59.073683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-16 20:40:59.073763: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-16 20:40:59.073839: W tensorflow/c

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 3ms/step - loss: 0.0822 - mean_squared_error: 0.0822
Loss: 0.08218719810247421 Tiempo transcurrido: 134.0812656879425

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 4ms/step - loss: 0.1052 - mean_squared_error: 0.1052
Loss: 0.10518810153007507 Tiempo transcurrido: 67.25581765174866

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.1810 - mean_squared_error: 0.1810
Loss: 0.18097840249538422 Tiempo transcurrido: 82.98474740982056

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 17.2866 - mean_squared_error: 17.2866
Loss: 17.28662109375 T

5/5 [==============================] - 0s 2ms/step - loss: 0.0367 - mean_squared_error: 0.0367
Loss: 0.03671078383922577 Tiempo transcurrido: 26.36995792388916

--- Starting trial: run-32
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 3ms/step - loss: 0.0348 - mean_squared_error: 0.0348
Loss: 0.034772079437971115 Tiempo transcurrido: 151.2824993133545

--- Starting trial: run-33
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 4ms/step - loss: 0.0346 - mean_squared_error: 0.0346
Loss: 0.0346190445125103 Tiempo transcurrido: 81.46939182281494

--- Starting trial: run-34
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 3ms/step - loss: 0.0520 - mean_squared_error: 0.0520
Loss: 0.05198704078793526 Tiempo transcurrido: 83.52090430259705

--- Starting trial: run-35
{'layers': 2, 'num_units': 50

5/5 [==============================] - 0s 2ms/step - loss: 0.0333 - mean_squared_error: 0.0333
Loss: 0.03325919434428215 Tiempo transcurrido: 29.045179843902588

--- Starting trial: run-63
{'layers': 2, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0584 - mean_squared_error: 0.0584
Loss: 0.058366142213344574 Tiempo transcurrido: 20.797242879867554

--- Starting trial: run-64
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0420 - mean_squared_error: 0.0420
Loss: 0.04197508096694946 Tiempo transcurrido: 90.0740578174591

--- Starting trial: run-65
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0359 - mean_squared_error: 0.0359
Loss: 0.035898055881261826 Tiempo transcurrido: 48.04292941093445

--- Starting trial: run-66
{'layers': 3, 'num_units'

5/5 [==============================] - 0s 2ms/step - loss: 0.0315 - mean_squared_error: 0.0315
Loss: 0.031494949012994766 Tiempo transcurrido: 55.0037055015564

--- Starting trial: run-94
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0451 - mean_squared_error: 0.0451
Loss: 0.04506915807723999 Tiempo transcurrido: 29.227661609649658

--- Starting trial: run-95
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.0296 - mean_squared_error: 0.0296
Loss: 0.029581965878605843 Tiempo transcurrido: 18.610822677612305

--- Starting trial: run-96
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0331 - mean_squared_error: 0.0331
Loss: 0.03307526558637619 Tiempo transcurrido: 73.5267436504364

--- Starting trial: run-97
{'layers': 4, 'num_units':

5/5 [==============================] - 0s 2ms/step - loss: 0.1027 - mean_squared_error: 0.1027
Loss: 0.10273478180170059 Tiempo transcurrido: 138.78371787071228

--- Starting trial: run-125
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0388 - mean_squared_error: 0.0388
Loss: 0.038817569613456726 Tiempo transcurrido: 66.5687096118927

--- Starting trial: run-126
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0350 - mean_squared_error: 0.0350
Loss: 0.035041678696870804 Tiempo transcurrido: 33.416282653808594

--- Starting trial: run-127
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0635 - mean_squared_error: 0.0635
Loss: 0.06350080668926239 Tiempo transcurrido: 17.405747890472412
128


In [14]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [15]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,3,200,0.001,16,0.029582,18.610823
1,4,100,0.001,16,0.033464,17.276093


In [16]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 12.816 minutes


In [17]:
df["Tiempo de ejecución"]

0     18.610823
1     17.276093
2     19.687886
3     22.090298
4     20.511935
5     17.093636
6     26.369958
7     20.337040
8     25.329070
9     26.235701
10    15.956948
11    18.008683
12    15.865189
13    42.159092
14    27.363442
15    17.815203
16    15.378498
17    19.113473
18    26.069265
19    20.484787
20    20.797243
21    17.405748
22    19.372958
23    42.462574
24    41.887536
25    16.035051
26    16.677071
27    42.383313
28    27.362544
29    42.320909
30    26.964505
31    23.534415
Name: Tiempo de ejecución, dtype: float64